In [1]:
import numpy as np, pandas as pd
import matplotlib.pyplot as plt, seaborn as sns

import networkx as nx

## 그래프 생성

In [2]:
network_df = pd.DataFrame(
    {
        'sender' : [1, 2, 3, 4, 5, 6, 7, 8, 8, 8], 
        'receiver' : [3, 5, 4, 2, 3, 8, 8, 6, 7, 6], 
        'send_money' : [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 
        'recent_trx' : [1, 5, 2, 4, 6, 5, 1, 6, 7, 8], 
        'ek' : [11, 12, 13, 14, 15, 16, 17, 18, 19, 20]
    }
)
network_df

,sender,receiver,send_money,recent_trx,ek
0,1,3,10,1,11
1,2,5,20,5,12
2,3,4,30,2,13
3,4,2,40,4,14
4,5,3,50,6,15
5,6,8,60,5,16
6,7,8,70,1,17
7,8,6,80,6,18
8,8,7,90,7,19
9,8,6,100,8,20


In [3]:
fraud_df = pd.DataFrame(
    {
        'sender' : [1, 2, 3, 4, 5, 6, 7, 8], 
        'fraud' : [1, 0, 0, 0, 0, 0, 0, 1]
    }
)
fraud_df

,sender,fraud
0,1,1
1,2,0
2,3,0
3,4,0
4,5,0
5,6,0
6,7,0
7,8,1


In [4]:
#그래프 생성
G = nx.from_pandas_edgelist(network_df, 
                            source = 'sender', #송금 지점
                            target = 'receiver', #수금 지점
                            edge_attr = ['send_money', 'recent_trx'], #에지에 부여할 특성 
                            edge_key = 'ek', #에지 데이터프레임의 키. 
                            create_using = nx.MultiDiGraph)

In [5]:
# node의 fraud 특성을 그래프에 부여
fraud_dict = fraud_df.set_index('sender')['fraud'].to_dict()
nx.set_node_attributes(G, fraud_dict, 'fraud')

In [6]:
G.nodes()

NodeView((1, 3, 2, 5, 4, 6, 8, 7))

In [7]:
G.edges()

OutMultiEdgeDataView([(1, 3), (3, 4), (2, 5), (5, 3), (4, 2), (6, 8), (8, 6), (8, 6), (8, 7), (7, 8)])

In [8]:
G.edges[1, 3, 11] # 1 : 송금, 3 : 수금, 11 : 에지 키 

{'send_money': 10, 'recent_trx': 1}

## 그래프 시각화

In [9]:
import plotly.graph_objects as go

In [10]:
# Define colors based on fraud attribute
color_map = {0: '#004b99', 1: '#ef833c'}

In [11]:
from pyvis.network import Network

In [12]:
# Create the pyvis network
net = Network(directed=True, notebook=True)

# Add nodes with color based on fraud attribute
for node in G.nodes:
    net.add_node(node, color=color_map[G.nodes[node]['fraud']], title=f"Node {node}, Fraud: {G.nodes[node]['fraud']}")

# Add edges with weight proportional to send_money
for u, v, data in G.edges(data=True):
    weight = data['send_money']
    net.add_edge(u, v, value=weight, title=f"Send Money: {weight}")

# Generate and show the network
net.show("interactive_multidigraph.html")

interactive_multidigraph.html
